In [1]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, FunctionTool
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
import asyncio
import os
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Define storage directory
storage_dir = "../.index_storage"

# Check if index already exists
if os.path.exists(storage_dir):
    print("Loading existing index...")
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    index = load_index_from_storage(storage_context)
else:
    print("Creating new index...")
    documents = SimpleDirectoryReader("../data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=storage_dir)

Loading existing index...
Loading llama_index.core.storage.kvstore.simple_kvstore from ../.index_storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ../.index_storage/index_store.json.


In [5]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=index.as_query_engine(),
        name="document_search",
        description="Luôn luôn sử dụng công cụ này để tìm kiếm thông tin trong sách giáo khoa cấp trung học phổ thông.",
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    llm=OpenAI(model="gpt-4o-mini")
)

In [14]:
query_engine.query(
    """
Nếu định luật I Newton đúng thì tại sao các vật chuyển động trên mặt đất đều dừng lại?

A. vì có ma sát.

B. vì các vật không phải là chất điểm.

C. vì có lực hút của Trái Đất.

D. vì do Trái Đất luôn luôn chuyển động.
    """
)

Generated 5 sub questions.


Response(response='A. vì có ma sát.', source_nodes=[NodeWithScore(node=TextNode(id_='2ce0af86-6484-4f28-9acf-1c2fe126135e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text="Sub question: What is Newton's First Law of Motion and how does it relate to objects in motion?\nResponse: Newton's First Law of Motion states that an object will remain at rest or continue to move at a constant velocity unless acted upon by an external force. This law explains how objects in motion will stay in motion and objects at rest will stay at rest unless a force is applied to change their state of motion.", mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None), NodeWithScore(node=TextNode(id_='73e2e486-6c27-47c6-8830-0b6f9ee9f65c', embedding=None, metadata={}, excluded_embed_metadata_keys=[